##匯入資料

In [ ]:
filepath = '/content/MyGoogleDrive/My Drive/'
from google.colab import drive
drive.mount('/content/MyGoogleDrive')
import pandas as pd
df_original=pd.read_csv(filepath + "credit_card_balance.csv")

In [ ]:
df=df_original

##月份補成正值 並刪除我不需要的資料

In [ ]:
df.MONTHS_BALANCE.abs() #月份補成正值    #刪除我不要的東東
df=df.drop(['AMT_DRAWINGS_OTHER_CURRENT','CNT_DRAWINGS_OTHER_CURRENT','AMT_RECIVABLE','AMT_RECEIVABLE_PRINCIPAL','SK_DPD_DEF','AMT_PAYMENT_CURRENT'],axis=1)

In [ ]:
df.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD
0,2562384,378907,-6,56.970,135000,0.0,877.5,877.5,1700.325,1800.0,0.000,0.0,1,1.0,35.0,Active,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,2250.000,2250.0,64875.555,1.0,1,0.0,69.0,Active,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,2250.000,2250.0,31460.085,0.0,0,0.0,30.0,Active,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,11795.760,11925.0,233048.970,1.0,1,0.0,10.0,Active,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,11547.0,22924.890,27000.0,453919.455,0.0,1,1.0,101.0,Active,0


##把SK_ID_CURR & SK_ID_PREV 變成numpy陣列(方便迴圈)

In [ ]:
#groupby迴圈版 用SK_ID_CURR的值
value=(df['SK_ID_CURR'].drop_duplicates())
members=value.sort_values()
members.reset_index(drop=True)
members=members.to_numpy()
members

array([100006, 100011, 100013, ..., 456247, 456248, 456250])

##處理AMT_BALANCE 找出重複貸款的人

In [ ]:
# 這邊出了點問題
x=[]
for j in members:
  LOAD=df[['AMT_BALANCE','MONTHS_BALANCE']][(df['SK_ID_CURR']==j)].sort_values(by=["MONTHS_BALANCE"],ascending=False).reset_index() #sort_values(by=[]) by只能用在DataFrame
  LOAD=LOAD.drop(['MONTHS_BALANCE',"index"],axis=1) 
  LOAD_T=LOAD['AMT_BALANCE'][0]
  AMT_BALANCE_count=0
  for i in LOAD['AMT_BALANCE']:
    if i < LOAD_T: 
      AMT_BALANCE_count+=1
    LOAD_T=i
  x.append(AMT_BALANCE_count)
  print(AMT_BALANCE_count)
AMT_BALANCE_count=pd.DataFrame(x,columns=['AMT_BALANCE_count'])
AMT_BALANCE_count=AMT_BALANCE_count.set_index(members)
AMT_BALANCE_count.index.name='SK_ID_CURR'
AMT_BALANCE_count #直接concat


##groupby即可運算的columns

In [ ]:
def fillna1(dfna):
  df[dfna]=df[dfna].fillna(0)
  
  return df[dfna]
list=['AMT_DRAWINGS_ATM_CURRENT','CNT_DRAWINGS_ATM_CURRENT','AMT_DRAWINGS_CURRENT','CNT_DRAWINGS_CURRENT','AMT_DRAWINGS_POS_CURRENT','CNT_DRAWINGS_POS_CURRENT']
for i in list:
  fillna1(i)

In [ ]:
df1=df.groupby(['SK_ID_CURR']).aggregate({'SK_ID_PREV':'max',
                     'AMT_BALANCE':'max',
                     'AMT_CREDIT_LIMIT_ACTUAL':'max',
                     'CNT_INSTALMENT_MATURE_CUM':'max',
                     'AMT_DRAWINGS_ATM_CURRENT':'sum',
                     'CNT_DRAWINGS_ATM_CURRENT':'sum',
                     'AMT_DRAWINGS_CURRENT':'sum',
                     'CNT_DRAWINGS_CURRENT':'sum',
                     'AMT_DRAWINGS_POS_CURRENT':'sum',
                     'CNT_DRAWINGS_POS_CURRENT':'sum',
                     'SK_DPD':'max'
                     })
df1

,SK_ID_PREV,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,CNT_INSTALMENT_MATURE_CUM,AMT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,CNT_DRAWINGS_CURRENT,AMT_DRAWINGS_POS_CURRENT,CNT_DRAWINGS_POS_CURRENT,SK_DPD
SK_ID_CURR,,,,,,,,,,,
100006,1489396,0.000,270000,0.0,0.0,0.0,0.000,0,0.000,0.0,0
100011,1843384,189000.000,180000,33.0,180000.0,4.0,180000.000,4,0.000,0.0,0
100013,2038692,161420.220,157500,22.0,571500.0,23.0,571500.000,23,0.000,0.0,1
100021,2594025,0.000,675000,0.0,0.0,0.0,0.000,0,0.000,0.0,0
100023,1499902,0.000,225000,0.0,0.0,0.0,0.000,0,0.000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
456244,2181926,453627.675,450000,17.0,1003500.0,43.0,1100537.910,56,96883.650,13.0,0
456246,1079732,43490.115,135000,7.0,0.0,0.0,121594.050,20,121594.050,20.0,0
456247,1595171,190202.130,180000,32.0,202950.0,11.0,204203.115,14,1253.115,3.0,1


###重新定義索引並跑回圈找出三個可以用的平均值

In [ ]:
df1=df1.reset_index()

In [ ]:
DRAWINGS_ATM_average=[]
DRAWINGS_average=[]
DRAWINGS_POS_average=[]

for i in range(103558):
  DRAWINGS_ATM_average.append(df1['AMT_DRAWINGS_ATM_CURRENT'][i]/df1['CNT_DRAWINGS_ATM_CURRENT'][i])
  DRAWINGS_average.append(df1['AMT_DRAWINGS_CURRENT'][i]/df1['CNT_DRAWINGS_CURRENT'][i])
  DRAWINGS_POS_average.append(df1['AMT_DRAWINGS_POS_CURRENT'][i]/df1['CNT_DRAWINGS_POS_CURRENT'][i])



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


In [ ]:
#迴圈跑完多三個list 把原本的columns刪掉
df1=df1.drop(['AMT_DRAWINGS_ATM_CURRENT','CNT_DRAWINGS_ATM_CURRENT','AMT_DRAWINGS_CURRENT','CNT_DRAWINGS_CURRENT','AMT_DRAWINGS_POS_CURRENT','CNT_DRAWINGS_POS_CURRENT'],axis=1)

In [ ]:
#補上我要的三個list
df1['DRAWINGS_ATM_average']=DRAWINGS_ATM_average
df1['DRAWINGS_POS_average']=DRAWINGS_POS_average
df1['DRAWINGS_average']=DRAWINGS_average

In [ ]:
#把他們的空值補成0
list1=['DRAWINGS_ATM_average','DRAWINGS_POS_average','DRAWINGS_average']
def fillna1(dfna):
  df1[dfna]=df1[dfna].fillna(0)
  
  return df1[dfna]
for i in list1:
  fillna1(i)
df1

,SK_ID_CURR,SK_ID_PREV,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,CNT_INSTALMENT_MATURE_CUM,SK_DPD,DRAWINGS_ATM_average,DRAWINGS_POS_average,DRAWINGS_average
0,100006,1489396,0.000,270000,0.0,0,0.000000,0.000000,0.000000
1,100011,1843384,189000.000,180000,33.0,0,45000.000000,0.000000,45000.000000
2,100013,2038692,161420.220,157500,22.0,1,24847.826087,0.000000,24847.826087
3,100021,2594025,0.000,675000,0.0,0,0.000000,0.000000,0.000000
4,100023,1499902,0.000,225000,0.0,0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
103553,456244,2181926,453627.675,450000,17.0,0,23337.209302,7452.588462,19652.462679
103554,456246,1079732,43490.115,135000,7.0,0,0.000000,6079.702500,6079.702500
103555,456247,1595171,190202.130,180000,32.0,1,18450.000000,417.705000,14585.936786
103556,456248,2743495,0.000,900000,0.0,0,0.000000,0.000000,0.000000


In [ ]:
df1.set_index('SK_ID_CURR',inplace=True)

In [ ]:
df1

,SK_ID_PREV,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,CNT_INSTALMENT_MATURE_CUM,SK_DPD,DRAWINGS_ATM_average,DRAWINGS_POS_average,DRAWINGS_average
SK_ID_CURR,,,,,,,,
100006,1489396,0.000,270000,0.0,0,0.000000,0.000000,0.000000
100011,1843384,189000.000,180000,33.0,0,45000.000000,0.000000,45000.000000
100013,2038692,161420.220,157500,22.0,1,24847.826087,0.000000,24847.826087
100021,2594025,0.000,675000,0.0,0,0.000000,0.000000,0.000000
100023,1499902,0.000,225000,0.0,0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
456244,2181926,453627.675,450000,17.0,0,23337.209302,7452.588462,19652.462679
456246,1079732,43490.115,135000,7.0,0,0.000000,6079.702500,6079.702500
456247,1595171,190202.130,180000,32.0,1,18450.000000,417.705000,14585.936786


In [ ]:
#信用卡額度取眾數
LIMIT_mode=df.groupby('SK_ID_CURR')['AMT_CREDIT_LIMIT_ACTUAL'].agg(lambda x: x.value_counts().index[0]).reset_index()
LIMIT_mode=LIMIT_mode.drop(['SK_ID_CURR'],axis=1) 
LIMIT_mode=LIMIT_mode.set_index(members) 
LIMIT_mode.index.name='SK_ID_CURR'
LIMIT_mode #可以直接concat

In [ ]:

# 注意要補空值ID 不然等於0也要放進去 去掉0找平均
dfi=df[df['AMT_INST_MIN_REGULARITY']!=0]
INST_MIN_mean=dfi.groupby('SK_ID_CURR')['AMT_INST_MIN_REGULARITY'].mean()
type(INST_MIN_mean)
INST_MIN_mean=INST_MIN_mean.to_frame()
INST_MIN_mean #直接concat

,AMT_INST_MIN_REGULARITY
SK_ID_CURR,
100011,8751.642273
100013,5884.273636
100028,7360.036714
100042,4393.404231
100043,17482.628250
...,...
456242,7977.317143
456244,15327.529412
456246,1644.743571


In [ ]:
#去掉0找平均 
dfp=df[df['AMT_PAYMENT_TOTAL_CURRENT']!=0]
AMT_PAYMENT_TOTAL_CURRENT_mean=dfp.groupby('SK_ID_CURR')['AMT_PAYMENT_TOTAL_CURRENT'].mean()
AMT_PAYMENT_TOTAL_CURRENT_mean=AMT_PAYMENT_TOTAL_CURRENT_mean.to_frame()
AMT_PAYMENT_TOTAL_CURRENT_mean #直接concat

,AMT_PAYMENT_TOTAL_CURRENT
SK_ID_CURR,
100011,10452.656250
100013,43629.903000
100028,6700.035732
100042,16731.486486
100043,29572.029000
...,...
456242,22377.323571
456244,38329.781143
456246,20739.120000


## 合併表格

In [ ]:
ndf1=pd.concat([df1,AMT_PAYMENT_TOTAL_CURRENT_mean,INST_MIN_mean,LIMIT_mode,AMT_BALANCE_count],axis=1)

In [ ]:
ndf1=ndf1.rename(columns={'AMT_BALANCE':'AMT_BALANCE_max','AMT_CREDIT_LIMIT_ACTUAL':'AMT_CREDIT_LIMIT_ACTUAL_max','CNT_INSTALMENT_MATURE_CUM':'CNT_INSTALMENT_MATURE_CUM_max','SK_DPD':'SK_DPD_max','AMT_PAYMENT_TOTAL_CURRENT':'AMT_PAYMENT_TOTAL_CURRENT_mean','AMT_INST_MIN_REGULARITY':'AMT_INST_MIN_REGULARITY_mean','AMT_CREDIT_LIMIT_ACTUAL':'AMT_CREDIT_LIMIT_ACTUAL_mode'})

In [ ]:
ndf1.columns=['SK_ID_PREV','AMT_BALANCE_max','AMT_CREDIT_LIMIT_ACTUAL_max','CNT_INSTALMENT_MATURE_CUM_max','SK_DPD_max','DRAWINGS_ATM_average','DRAWINGS_POS_average','DRAWINGS_average','AMT_PAYMENT_TOTAL_CURRENT_mean','AMT_INST_MIN_REGULARITY_mean','AMT_CREDIT_LIMIT_ACTUAL_mode','AMT_BALANCE_count']

In [ ]:
ndf1

,SK_ID_PREV,AMT_BALANCE_max,AMT_CREDIT_LIMIT_ACTUAL_max,CNT_INSTALMENT_MATURE_CUM_max,SK_DPD_max,DRAWINGS_ATM_average,DRAWINGS_POS_average,DRAWINGS_average,AMT_PAYMENT_TOTAL_CURRENT_mean,AMT_INST_MIN_REGULARITY_mean,AMT_CREDIT_LIMIT_ACTUAL_mode,AMT_BALANCE_count
SK_ID_CURR,,,,,,,,,,,,
100006,1489396,0.000,270000,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,270000,0
100011,1843384,189000.000,180000,33.0,0,45000.000000,0.000000,45000.000000,10452.656250,8751.642273,180000,0
100013,2038692,161420.220,157500,22.0,1,24847.826087,0.000000,24847.826087,43629.903000,5884.273636,157500,6
100021,2594025,0.000,675000,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,675000,0
100023,1499902,0.000,225000,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,225000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
456244,2181926,453627.675,450000,17.0,0,23337.209302,7452.588462,19652.462679,38329.781143,15327.529412,450000,10
456246,1079732,43490.115,135000,7.0,0,0.000000,6079.702500,6079.702500,20739.120000,1644.743571,135000,3
456247,1595171,190202.130,180000,32.0,1,18450.000000,417.705000,14585.936786,24438.026250,4479.898500,180000,6


In [ ]:
ndf1=ndf1.fillna(0)

## 輸出表格

In [ ]:
ndf1.to_csv(filepath+'new_credit_card_balance.csv')